# test-models.ipynb

Just test model usage

In [1]:
import os, sys
from loguru import logger
sys.path.append(os.path.join(".."))
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

/home/johann/software/miniconda3/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        dict(llm="openai/o1"),
        dict(llm="openai/o1-mini"),
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-latest"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        dict(llm="xai/grok-2"),
        dict(llm="xai/grok-beta"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_AIP_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
    )
)
update_llm_config(config)
llms = LLMS(config)
config

{'llms': [{'llm': 'openai/gpt-4o',
   'api_key_env': 'MY_OPENAI_API_KEY',
   'alias': 'openai/gpt-4o'},
  {'llm': 'openai/gpt-4o-mini',
   'api_key_env': 'MY_OPENAI_API_KEY',
   'alias': 'openai/gpt-4o-mini'},
  {'llm': 'openai/o1',
   'api_key_env': 'MY_OPENAI_API_KEY',
   'alias': 'openai/o1'},
  {'llm': 'openai/o1-mini',
   'api_key_env': 'MY_OPENAI_API_KEY',
   'alias': 'openai/o1-mini'},
  {'llm': 'gemini/gemini-2.0-flash-exp',
   'api_key_env': 'MY_GEMINI_API_KEY',
   'alias': 'gemini/gemini-2.0-flash-exp'},
  {'llm': 'gemini/gemini-1.5-flash',
   'api_key_env': 'MY_GEMINI_API_KEY',
   'alias': 'gemini/gemini-1.5-flash'},
  {'llm': 'gemini/gemini-1.5-pro',
   'api_key_env': 'MY_GEMINI_API_KEY',
   'alias': 'gemini/gemini-1.5-pro'},
  {'llm': 'anthropic/claude-3-5-sonnet-latest',
   'api_key_env': 'MY_ANTHROPIC_API_KEY',
   'alias': 'anthropic/claude-3-5-sonnet-latest'},
  {'llm': 'anthropic/claude-3-opus-20240229',
   'api_key_env': 'MY_ANTHROPIC_API_KEY',
   'alias': 'anthropic/

In [5]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'openai/o1',
 'openai/o1-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-latest',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-2',
 'xai/grok-beta']

In [19]:
for llmalias in llms.list_aliases():
    print(f"{llmalias}:")
    try:
        c1, c2 = llms.cost_per_token(llmalias)
        print(f"  Cost per token, input={c1}, output={c2}")
        mt = llms.max_prompt_tokens(llmalias)
        print(f"  Maximum tokens: {mt}")  
    except:
        print(f"  No information")

openai/gpt-4o:
  Cost per token, input=2.5e-06, output=1e-05
  Maximum tokens: 16384
openai/gpt-4o-mini:
  Cost per token, input=1.5e-07, output=6e-07
  Maximum tokens: 16384
openai/o1:
  Cost per token, input=1.5e-05, output=6e-05
  Maximum tokens: 100000
openai/o1-mini:
  Cost per token, input=3e-06, output=1.2e-05
  Maximum tokens: 65536
gemini/gemini-2.0-flash-exp:
  Cost per token, input=0.0, output=0
  Maximum tokens: 8192
gemini/gemini-1.5-flash:
  Cost per token, input=7.5e-08, output=3e-07
  Maximum tokens: 8192
gemini/gemini-1.5-pro:
  Cost per token, input=3.5e-06, output=1.05e-05
  Maximum tokens: 8192
anthropic/claude-3-5-sonnet-latest:
  No information
anthropic/claude-3-opus-20240229:
  Cost per token, input=1.5e-05, output=7.5e-05
  Maximum tokens: 4096
mistral/mistral-large-latest:
  Cost per token, input=2e-06, output=6e-06
  Maximum tokens: 128000
xai/grok-2:
  No information
xai/grok-beta:
  Cost per token, input=5e-06, output=1.5e-05
  Maximum tokens: 131072


In [6]:
messages = llms.make_messages(query="What is the first name of the famous physicist who came up with the theory of relativity? Answer with the first name only.")


for llmalias in llms.list_aliases():
    print("Model:", llmalias)
    cinfo = {}
    ret = llms.query(
        llmalias, 
        messages=messages, 
        return_cost=True,
        debug=False,
    )
    answer = ret["answer"]
    error = ret["error"]
    if error:
        print("  ERROR:", error)
    else:
        print("  Answer:", answer)


Model: openai/gpt-4o

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

  ERROR: litellm.BadRequestError: OpenAIException - Error code: 400 - {'error': {'message': 'Unrecognized request argument supplied: cost', 'type': 'invalid_request_error', 'param': None, 'code': None}}
Model: openai/gpt-4o-mini

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

  ERROR: litellm.BadRequestError: OpenAIException - Error code: 400 - {'error': {'message': 'Unrecognized request argument supplied: cost', 'type': 'invalid_request_error', 'param': None, 'code': None}}
Model: openai/o1

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

  ERROR: litellm.NotFoundError: OpenAIException - Error code: 404 - {'e